# **Audio Classification with LSTM and Torch Audio**

## **Libraries Required**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
import torchaudio
import pandas as pd
import torch.nn.functional as F

## **About Dataset**

This dataset contains 8732 labeled sound excerpts ( <= 4s ) of urban sounds from 10 classes :`air_conditioner`, `car_horn`, `children_playing`, `dog_bark`, `drilling`, `engine_idling`, `gun_shot`, `jackhammer`, `siren` and `street_music`. The classes are drawn from the urban sound taxonomy.

In [44]:
class UrbanSoundDataset(Dataset):
    # Wrapper for the UrbanSound8K dataset
    # Argument List
    # path to the UrbanSound8K csv file
    # path to the UrbanSound8K audio files
    # list of folders to use in the dataset

    def __init__(self, csv_path, file_path, folderList):
        csvData = pd.read_csv(csv_path)
        # initialize lists to hold file names, labels, and folder numbers
        self.file_names = []
        self.labels = []
        self.folders = []
        # loop through the csv entries and only add entries from folders in the folder list
        for i in range(0, len(csvData)):
            if csvData.iloc[i, 5] in folderList:
                self.file_names.append(csvData.iloc[i, 0]) # extracting & appending each element from column 1- filenames
                self.labels.append(csvData.iloc[i, 6]) # extracting & appending each element from column 7 - classID
                self.folders.append(csvData.iloc[i, 5]) # extracting & appending elements from column 6 -folderNumber

        self.file_path = file_path
        self.folderList = folderList

    def __getitem__(self, index):
        # format the file path and load the file
        path = self.file_path + "fold" + str(self.folders[index]) + "/" + self.file_names[index]
        sound, sample_rate = torchaudio.load(path, normalize=True)
        soundData = torch.mean(sound, dim=0, keepdim=True)
        tempData = torch.zeros([1, 160000])  # tempData accounts for audio clips that are too short

        if soundData.numel() < 160000: # checks the number of elements in soundData tensor
            tempData[:, :soundData.numel()] = soundData
        else:
            tempData = soundData[:, :160000]

        soundData = tempData

        mel_specgram = torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate)(soundData)  # (channel, n_mels, time)
        mel_specgram_norm = (mel_specgram - mel_specgram.mean()) / mel_specgram.std()
        mfcc = torchaudio.transforms.MFCC(sample_rate=sample_rate)(soundData)  # (channel, n_mfcc, time)
        mfcc_norm = (mfcc - mfcc.mean()) / mfcc.std()
        
        feature = torch.cat([mel_specgram, mfcc], axis=1)
        
        return feature[0].permute(1, 0), self.labels[index]

    def __len__(self):
        return len(self.file_names)

In [37]:
class AudioLSTM(nn.Module):

    def __init__(self, n_feature=5, out_feature=5, n_hidden=256, n_layers=2, drop_prob=0.5):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.n_feature = n_feature

        self.lstm = nn.LSTM(self.n_feature, self.n_hidden, self.n_layers, dropout=self.drop_prob, batch_first=True)

        self.dropout = nn.Dropout(drop_prob)

        self.fc = nn.Linear(n_hidden, out_feature)

    def forward(self, x, hidden):
        # x.shape (batch, seq_len, n_features)
        l_out, l_hidden = self.lstm(x, hidden)

        # out.shape (batch, seq_len, n_hidden*direction)
        out = self.dropout(l_out)

        # out.shape (batch, out_feature)
        out = self.fc(out[:, -1, :])

        # return the final output and the hidden state
        return out, l_hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data

        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        return hidden

In [38]:
def train(model, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)

        model.zero_grad()
        output, hidden_state = model(data, model.init_hidden(hyperparameters["batch_size"]))
        
        loss = criterion(output, target)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        if batch_idx % log_interval == 0: #print training stats
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))

In [39]:
def test(model, epoch):
    model.eval()
    correct = 0
    y_pred, y_target = [], []
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        
        output, hidden_state = model(data, model.init_hidden(hyperparameters["batch_size"]))
        
        pred = torch.max(output, dim=1).indices
        correct += pred.eq(target).cpu().sum().item()
        y_pred = y_pred + pred.tolist()
        y_target = y_target + target.tolist()
        
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [40]:
hyperparameters = {
    "lr": 0.01, 
    "weight_decay": 0.0001,
    "batch_size": 128, 
    "in_feature": 168, # number of features per timesteps (e.g., MFCCs, spectrogram bins)
    "out_feature": 10 # number of output units ( e.g., number of classes )
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

csv_path =  '../data/urbansound8k/UrbanSound8K.csv'
file_path = '../data/urbansound8k/'

train_set = UrbanSoundDataset(csv_path, file_path, range(1, 10))
test_set = UrbanSoundDataset(csv_path, file_path, [10])
print("Train set size: " + str(len(train_set)))
print("Test set size: " + str(len(test_set)))

cuda
Train set size: 7895
Test set size: 837


1. **num_workers**

    * What it means: Nu mber of subprocesses to use for data loading.
    * `num_workers=0`: data loading is done in the main process (slower).
    * `num_workers>0`: multiple worker processes load data in parallel (faster).

👉 Here it’s set to 1, meaning one extra worker process will load data in parallel to the training loop.

2. **pin_memory**

    * What it means: If True, the data loader will copy tensors into page-locked (pinned) memory.

    * Why? When using a GPU (device == 'cuda'), pinned memory allows faster and more efficient transfer of data from CPU → GPU.

    * Without it, transfers may be slower because regular memory can be swapped out by the OS.

👉 It’s only useful when training on GPU. That’s why it’s conditionally set only if device == 'cuda'.

In [45]:
kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {}  # needed for using datasets on gpu

train_loader = torch.utils.data.DataLoader(train_set, batch_size=hyperparameters["batch_size"], shuffle=True, drop_last=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=hyperparameters["batch_size"], shuffle=True, drop_last=True, **kwargs)

In [42]:
model = AudioLSTM(
    n_feature=hyperparameters["in_feature"],
    out_feature=hyperparameters["out_feature"])

model.to(device)
print(model)

optimizer = optim.Adam(
    model.parameters(),
    lr=hyperparameters['lr'],
    weight_decay=hyperparameters['weight_decay'])

criterion = nn.CrossEntropyLoss()
clip = 5  # gradient clipping

log_interval = 10

AudioLSTM(
  (lstm): LSTM(168, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=256, out_features=10, bias=True)
)


In [ ]:
for epoch in range(1, 101):
    train(model, epoch)
    test(model, epoch)

/home/sebdui/learn/.learnML/lib/python3.12/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/home/sebdui/learn/.learnML/lib/python3.12/site-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 

Train Epoch: 1 [0/7895 (0%)]	Loss: 0.599352
Train Epoch: 1 [1280/7895 (16%)]	Loss: 0.677366
Train Epoch: 1 [2560/7895 (33%)]	Loss: 0.722687
Train Epoch: 1 [3840/7895 (49%)]	Loss: 0.673987
Train Epoch: 1 [5120/7895 (66%)]	Loss: 0.638204
Train Epoch: 1 [6400/7895 (82%)]	Loss: 0.523225
Train Epoch: 1 [7680/7895 (98%)]	Loss: 0.698279
